In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import Clean_CSV

In [ ]:
import os
from torch import nn
import torch.utils.data as data
from torchvision import datasets, transforms
import torch.optim as optim
import torch

In [ ]:

cases_list = ['b03','b06','b09','bi001','bi01','bi1','da001','da01','da1']

cases_listmulti = ['b06pm','b09pm','bi001pm','bi01pm','da01pm']

# Loading, Cleaning Volume-Gamma-Nd dataframes

In [ ]:
## Volume-Gamma data for 3-drop cases

csv_file = ['RawData/' + elem + '_GVol.csv' for elem in cases_list]
csv_names = [elem + '_csv' for elem in cases_list]

for csv, name in zip(csv_file,csv_names):
    exec(f"{name} = pd.read_csv('{csv}')")
    print('dataframe loaded:' + f"'{name}'")

In [ ]:
## Volume-Gamma data for multi-drop cases

csv_filemulti = ['RawData/' + elem + '_GVol.csv' for elem in cases_listmulti]
csv_names_multi = [elem + '_csv' for elem in cases_listmulti]

for csv, name in zip(csv_filemulti,csv_names_multi):
    exec(f"{name} = pd.read_csv('{csv}')")
    print('dataframe loaded:' + f"'{name}'")


In [ ]:
## Dataframe cleanup

df_namescl = ['df_' + elem for elem in cases_list]

df_namescl_multi = ['df_' + elem for elem in cases_listmulti]

for dfname,csvname in zip(df_namescl,csv_names):
    exec(f"{dfname} = Clean_CSV.clean_csv({csvname},list({csvname}.columns.values)[1:3])")
    print('dataframe cleaned:' + f"'{dfname}'")
    
for dfname,csvname in zip(df_namescl_multi,csv_names_multi):
    exec(f"{dfname} = Clean_CSV.clean_csv({csvname},list({csvname}.columns.values)[1:3])")
    print('dataframe cleaned:' + f"'{dfname}'")



In [ ]:
## Number of drops data for 3-drop

Nd_names = ['Nd_' + elem for elem in csv_names]
Nd_csv_file = ['RawData/Nd/' + elem + '_dnum_corr.csv' for elem in cases_list]


for Nd,csv in zip(Nd_names,Nd_csv_file):
    df = pd.read_csv(csv)
    label_list = list(df.columns.values)
    df.rename(columns={label_list[0]: 'Ndrops'}, inplace=True)
    df['Time'] = df.apply(lambda row: row.name*0.005,axis=1)
    df = df[['Time','Ndrops']]
    exec(f"{Nd} = df")
    print('file loaded:' + f"'{Nd}'")


In [ ]:
## Number of drops data for multi-drop

Nd_names_multi = ['Nd_' + elem for elem in csv_names_multi]
Nd_csv_file_multi = ['RawData/Nd/' + elem + '_dnum_corr.csv' for elem in cases_listmulti]


for Nd,csv in zip(Nd_names_multi,Nd_csv_file_multi):
    df = pd.read_csv(csv)
    label_list = list(df.columns.values)
    df.rename(columns={label_list[0]: 'Ndrops'}, inplace=True)
    df['Time'] = df.apply(lambda row: row.name*0.005,axis=1)
    df = df[['Time','Ndrops']]
    exec(f"{Nd} = df")
    print('file loaded:' + f"'{Nd}'")


In [ ]:
Nd_b03_csv

# RNN-LSTM with PyTorch
## Training and testing number of drops for a single case

In [ ]:
timeseries = Nd_b03_csv[['Ndrops']].values.astype('float32')
plt.plot(timeseries)
plt.show()

In [ ]:
# OPTIONAL detrending and normalizing the linear trend
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

X = np.arange(len(timeseries)).reshape(-1, 1)
y = timeseries.reshape(-1, 1).astype('float32')

degree = 6
poly_features = PolynomialFeatures(degree=degree)
x_poly = poly_features.fit_transform(X.reshape(-1,1))

model = LinearRegression()
model.fit(x_poly, y)

trend = model.predict(x_poly).astype('float32')

detrended = y - trend.reshape(-1,1)

min_val = np.min(detrended)
max_val = np.max(detrended)

detrended = (detrended - min_val) / (max_val - min_val)

In [ ]:
plt.plot(trend)
plt.plot(y)
plt.show()

In [ ]:
plt.plot(detrended)
plt.show()

In [ ]:
# train-test split for time series
train_size = int(len(detrended) * 0.7)
test_size = len(detrended) - train_size
train, test = detrended[:train_size], detrended[train_size:]

In [ ]:
''' Transform the time series into a prediction 
dataset based on a given window. 
Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
'''
def create_dataset(dataset,lookback):
    X,y = [],[]
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback]
        target = dataset[i+1:i+lookback+1]
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)


In [ ]:
lookback = 10
X_train, y_train = create_dataset(train,lookback)
X_test, y_test = create_dataset(test,lookback)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

### LSTM Model class creation

In [ ]:
# Defining Hyperparameters
input_size = 1
hidden_size = 50
num_layers = 1
output_size = 1
num_epochs = 2000
learning_rate = 0.01

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self,input_size, hidden_size, output_size, num_layers):
        super(LSTMModel,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size,hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, output_size)
    def forward(self,x):
        out, _ = self.lstm(x)
        out = self.output_layer(out)
        return out

### Training LSTM with detrended/detrended data

In [ ]:
# Instance of the LSTM model
model = LSTMModel(input_size, hidden_size, output_size, num_layers)

# Setting loss function and optimizer
loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle = True, batch_size=8)

for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            loss = loss_func(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    if epoch % 100 != 0:
            continue
    model.eval()

    with torch.no_grad():
           y_pred = model(X_train)
           train_rmse = np.sqrt(loss_func(y_pred, y_train))
           y_pred = model(X_test)
           test_rmse = np.sqrt(loss_func(y_pred, y_test))
           
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))


In [ ]:
with torch.no_grad():
    # shift train predictions for plotting
    train_plot = np.ones_like(detrended) * np.nan
    y_pred = model(X_train)
    y_pred = y_pred[:, -1, :]
    train_plot[lookback:train_size] = model(X_train)[:, -1, :]
    # shift test predictions for plotting
    test_plot = np.ones_like(detrended) * np.nan
    test_plot[train_size+lookback:len(detrended)] = model(X_test)[:, -1, :]
# plot
plt.plot(detrended, c='b')
plt.plot(train_plot, c='r')
plt.plot(test_plot, c='g')
plt.show()